In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import os
import torch
import re
import string
import json
import ast
import matplotlib.pyplot as plt
import seaborn as sns

import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import torch.nn as nn
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
df=pd.read_csv('/kaggle/input/kriti2024/train.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.Categories.unique()

In [ ]:
df.isna().sum()

In [ ]:
# Define a function to extract i/ndividual categories from the string representation
def extract_categories(categories_string):
    # Safely evaluate the string as a list
    categories_list = ast.literal_eval(categories_string)
    return categories_list

# Apply the function to create a new column with lists of categories
df['Individual_Categories'] = df['Categories'].apply(extract_categories)

# Get unique individual categories
unique_categories = set()
for categories_list in df['Individual_Categories']:
    unique_categories.update(categories_list)

# Iterate through unique categories and add columns
for category in unique_categories:
    df[category] = df['Individual_Categories'].apply(lambda x: 1 if category in x else 0)

# Drop the original 'Categories' and 'Individual_Categories' columns
df = df.drop(['Categories', 'Individual_Categories','Id'], axis=1)

In [ ]:
df.head()

In [ ]:
df['Title_Abstract'] = df['Title'] + ' ' + df['Abstract']
df = df.drop(['Title','Abstract'], axis=1)
last_column = df.pop(df.columns[-1])
df.insert(0, last_column.name, last_column)
df.rename(columns={'Title_Abstract': 'Text'}, inplace=True)
df.head()

In [ ]:
df.Text =df.Text.str.lower()

In [ ]:
def remove_punct(text):
    punctuation =string.punctuation
    return text.translate(str.maketrans('' , '',punctuation))
df['Text']=df['Text'].apply(lambda x: remove_punct(x))

In [ ]:
from nltk.corpus import stopwords 
",".join(stopwords.words('english'))
STOPWRDS=set(stopwords.words('english'))
def remove_stp(Text):
    return " ".join([word for word in Text.split() if word not in STOPWRDS])
df['Text']=df['Text'].apply(lambda x: remove_stp(x))

In [ ]:
not_needed = [
     "using", "results", "performance", "show", "method", "approach",
    "also", "based", "problem", "proposed", "two", "information", "new", "large",
    "different", "study", "however",  "analysis", "one", "work", "used", "first",
    "set", "use", "existing", "present", "process", "demonstrate", "task", "general",
    "several", "due", "compared", "via", "moreover", "eg", "thus", "possible", "make",
    "like", "important", "key", "furthermore", "give", "state", "certain", "ie", "form",
    "allows", "finally", "often", "even", "many", "various", "well", "also", "however",
    "several", "due", "across", "may", "without", "among", "including", "particular",
    "especially", "either", "often", "even", "moreover", "thus", "ie", "eg", "although",
    "despite","would" ,"within" ,"",
]

import pandas as pd
def not_need(Text):
    return " ".join([word for word in Text.split() if word not in not_needed])

df['Text'] = df['Text'].apply(lambda x:not_need(x))

In [ ]:
df.Text[0]

In [ ]:
word_counts = df['Text'].apply(lambda x: len(x.split()))
min_words = word_counts.min()
max_words = word_counts.max()

print("Minimum number of words:", min_words)
print("Maximum number of words:", max_words)

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 6e-5
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df, test_size=0.13, random_state=42)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
target_cols = [col for col in df.columns if col not in ['Text']]
len(target_cols)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_dataset = BERTDataset(df_train, tokenizer, MAX_LEN)
test_dataset = BERTDataset(df_test, tokenizer, MAX_LEN)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
class MultiLabelSciBERT(torch.nn.Module):
    def __init__(self, num_labels):
        super(MultiLabelSciBERT, self).__init__()
        self.sci_bert = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
        self.classifier = torch.nn.Linear(768, num_labels)

    def forward(self, ids,mask):
        outputs = self.sci_bert(ids, attention_mask=mask)
        sequence_output = outputs[0][:, 0, :]  
        return self.classifier(sequence_output)

In [ ]:
model = MultiLabelSciBERT(57)
model.to(device);

In [ ]:
 model

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

In [ ]:
import torch
from torch.cuda.amp import autocast
import torch
from torch.cuda.amp import GradScaler

# Define the GradScaler object
scaler = GradScaler()
def train(epoch):
    model.train()
    for _, data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)

        optimizer.zero_grad()

        with autocast():
            outputs = model(ids, mask)
            loss = loss_fn(outputs, targets )

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if _ % 500 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')

In [ ]:
for epoch in range(EPOCHS):
    train(epoch) 

In [ ]:
torch.save(model.state_dict(), 'model_mid.pth')
# model_state_dict = torch.load('mode_mid.pth')
# model = MultiLabelSciBERT(57)
# model.load_state_dict(model_state_dict)

In [ ]:
import pandas as pd 

columns_to_check = ['q-bio.GN', 'q-fin.TR', 'q-bio.NC', 'q-fin.MF', 'q-fin.PM', 'cs.CE', 'q-bio.MN', 'math.AT',
                    'stat.CO', 'q-fin.EC', 'cs.OS', 'q-fin.CP', 'q-bio.CB', 'q-bio.BM', 'cs.DM', 'cs.GT', 'cs.AR',
                    'q-bio.TO', 'q-fin.PR', 'econ.GN', 'q-fin.GN', 'econ.TH', 'q-fin.RM']

mask = df[columns_to_check].any(axis=1)

filtered_df = df[mask]

filtered_df.head()

In [ ]:
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
filtered_df.shape

In [ ]:
train_dataset_minor = BERTDataset(filtered_df, tokenizer, MAX_LEN)

In [ ]:
minor_dataloader = DataLoader(train_dataset_minor, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True, pin_memory=True)

In [ ]:
def train(epoch):
    model.train()
    for _, data in enumerate(minor_dataloader, 0):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)

        optimizer.zero_grad()

        with autocast():
            outputs = model(ids, mask)
            loss = loss_fn(outputs, targets )

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if _ % 500 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')

In [ ]:
for epoch in range(EPOCHS):
    train(epoch) 

In [ ]:
torch.save(model.state_dict(), 'model_minor.pth')
# model_state_dict = torch.load('model_minor.pth')
# model.load_state_dict(model_state_dict)

In [ ]:
def validation():
    model.eval() 
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(test_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask)
#              outputs = model(ids, mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation()
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
test_df=pd.read_csv('/kaggle/input/kriti2024/test.csv')

In [ ]:
test_df.head()

In [ ]:
# Preprocess the text data
test_df['Text'] = test_df['Title'] + ' ' + test_df['Abstract']
test_df = test_df.drop(['Title', 'Abstract'], axis=1)
test_df['Text']=test_df['Text'].str.lower()
test_df['Text']=test_df['Text'].apply(lambda x: remove_stp(x))
test_df['Text'] = test_df['Text'].apply(lambda x:not_need(x))

In [ ]:
test_df.head()

In [ ]:
class BERTTestDatasets(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

# Create the test dataset
result_dataset = BERTTestDatasets(test_df, tokenizer, MAX_LEN)


In [ ]:
result_loader = DataLoader(result_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
def predict():
    model.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(result_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model(ids, mask)
            predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return predictions

# Get predictions
test_predictions = predict()

In [ ]:
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols)
submission_df.insert(0, 'Id', test_df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
submission_df.head()

In [ ]:
target_cols2 = [col for col in df.columns if col not in ['Text','Title','Abstract']]
len(target_cols2)

In [ ]:
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols2)
submission_df.insert(0, 'Id', test_df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
template_df = pd.read_csv('/kaggle/input/kritisample/sample_submission.csv')
required_column_order = template_df.columns.tolist()

In [ ]:
# Ensure that 'Id' is the first column
required_column_order.remove('Id')
required_column_order.insert(0, 'Id')

# Reorder your DataFrame columns
submission_df = submission_df[required_column_order]


In [ ]:
submission_df
submission_df.to_csv('/kaggle/working/submission_ordered_new.csv', index=False)

In [ ]:
submission_df.head()